In [12]:
import pandas as pd
import numpy as np
import nltk
import os
import pickle

from nltk.stem import PorterStemmer

In [18]:
# ps = PorterStemmer()
data = pd.read_csv("../Datasets/content_based_dataset_stems.csv")
data


,boardgame_id,boardgame_name,main_tags
0,30549,Pandemic,in pandem sever virul diseas have broken out s...
1,822,Carcassonne,carcassonn is a tileplac game in which the pla...
2,13,Catan,in catan formerli the settler of catan player ...
3,68448,7 Wonders,you are the leader of one of the 7 great citi ...
4,36218,Dominion,quotyou are a monarch like your parent befor y...
...,...,...,...
20583,16874,Vince Lombardi's Game,quotvinc lombardi gamequot is a footbal simul ...
20584,3589,Beast Wars Transformers Mutating Card Game,from the boxth beast war have begun choos side...
20585,296892,Sacred Rites,it ha come time to honor the tradit of the pas...
20586,10052,Bongo Kongo,each player take a posit adjac to one of the f...


In [1]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words="english")

In [8]:
test = ["One boardgame helps Two boardgame", 
        "Two boardgame help Four boardgame", 
        "Each boardgame helps many other boardgames at class."]

In [9]:
vector = cv.fit(test)
vector

CountVectorizer(max_features=5000, stop_words='english')

In [14]:
vector.vocabulary_

{'boardgame': 0, 'helps': 4, 'help': 3, 'boardgames': 1, 'class': 2}

In [16]:
pd.DataFrame(vector.transform(test).toarray())

,0,1,2,3,4
0,2,0,0,0,1
1,2,0,0,1,0
2,1,1,1,0,1


In [19]:
vector = cv.fit_transform(data["main_tags"])
vector

<20588x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 4860735 stored elements in Compressed Sparse Row format>

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
similary = cosine_similarity(vector)
similary

array([[1.        , 0.93025526, 0.93651672, ..., 0.40627318, 0.34666219,
        0.31444602],
       [0.93025526, 1.        , 0.95334654, ..., 0.40694715, 0.32550668,
        0.30566404],
       [0.93651672, 0.95334654, 1.        , ..., 0.41057804, 0.33834217,
        0.30685789],
       ...,
       [0.40627318, 0.40694715, 0.41057804, ..., 1.        , 0.14213381,
        0.13005122],
       [0.34666219, 0.32550668, 0.33834217, ..., 0.14213381, 1.        ,
        0.15249857],
       [0.31444602, 0.30566404, 0.30685789, ..., 0.13005122, 0.15249857,
        1.        ]])

In [37]:

def recommendation_system_cb(data, similary, boardgame, k):
    index  = data[data["boardgame_name"] == boardgame].index[0]
    distances = sorted(list(enumerate(similary[index])), reverse= True, key= lambda x : x[1])
    # print(distances)
    for distance in distances[1:k+1]:
        print(data.iloc[distance[0]]["boardgame_name"])

In [38]:
boardgame = "Marvel Champions: The Card Game"
recommendation_system_cb(data, similary, boardgame=boardgame, k=5)

The Lord of the Rings: The Card Game 0.9094899864777303
Sentinels of the Multiverse 0.8944677073647502
Legendary: A Marvel Deck Building Game 0.8912252932187208
Arkham Horror: The Card Game 0.8854481740924057
Android: Netrunner 0.913645778286805


In [11]:


# folder = "artifacts"
# if not os.path.exists(folder):
#     os.makedirs(folder)
#     print(f" Tạo thành công thư mục {folder} mới.")
# else:
#     os.rmdir(folder)
#     print(f" Xóa thành công thư mục {folder} hiện tại.")
#     os.makedirs(folder)
#     print(f" Tạo thành công thư mục {folder} mới.")



pickle.dump(data, open("artifacts/data_boardgame_content_based.pkl", "wb"))
pickle.dump(similary, open("artifacts/similarity_boardgame_content_based.pkl", "wb"))
pickle.dump(vector, open("artifacts/similarity_vector_tags_content_based.pkl", "wb"))